<a href="https://colab.research.google.com/github/Mariobahaa/Recommendation-System/blob/master/RS_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import gzip
import json
import numpy as np

# convert from json to pandas
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

# make dict and fit it by ids
def dict_ids(col):
  id = 0
  dic = dict() 
  for i in col:
    if i in dic:
      dic[i] = dic.get(i)
    else:
      dic[i] = id
      id += 1
  return dic

# make new col by mapping last ids to new ids
def new_col(col,dic):
  lis = list()
  for i in col:
    #if i in dic:
    lis.append(dic.get(i))
  return lis

#create products per user map
def productsOfUser(userId, productId):
  productsOfUserMap = dict()

  for i in range(0,len(userId)):
    productsOfUserMap[userId[i]] = np.array([]).astype(int)

  for i in range(0,len(userId)): 
    if productId[i] > -1:
      productsOfUserMap[userId[i]] = np.append(productsOfUserMap[userId[i]] ,productId[i])
        #productsOfUserMap[userId[i]].append([productId[i],cateId[i]])

  return productsOfUserMap

# def productsOfUser(userId, productId):
#   productsOfUserMap = dict()

#   for i in range(0,len(userId)): 
#     if productId[i] > -1:
#       productsOfUserMap[userId[i]].append(productId[i])
#         #productsOfUserMap[userId[i]].append([productId[i],cateId[i]])

#   return productsOfUserMap


In [0]:
df_all_beauty = getDF('/content/drive/My Drive/All Beauty/All_Beauty_5.json.gz')

userId = np.array(df_all_beauty['reviewerID'])
dict_all_beauty_users = dict_ids(userId)


productId = np.array(df_all_beauty['asin'])
dict_all_beauty_products = dict_ids(productId)

userId =new_col(userId,dict_all_beauty_users)
productId =new_col(productId,dict_all_beauty_products)

categoryId = np.ones(len(productId)).astype(int)

prodOfUser = productsOfUser(userId, productId)
cateOfUser = productsOfUser(userId, categoryId)

numberOfProducts = 0
numberOfCategories = 0

for i in prodOfUser:
  numberOfProducts += 1

for i in cateOfUser:
  numberOfCategories += 1

#print(prodOfUser)
# print(cateOfUser)
# print(categoryId)
#print(userId)
# print(df_all_beauty['userID'])
#print(df_all_beauty['reviewerID'])
# print(df_all_beauty['reviewerID'])
# print(productId)

In [0]:
# prodId = new_col(userId, prodOfUser)
# cateId = new_col(userId, cateOfUser)


userId = np.empty([0,1]).astype(int)


for i in prodOfUser:
  userId = np.append(userId, i)

#len(userId),1
prodId =[None]*len(userId)
cateId =[None]*len(userId)

# for i in range(0,len(userId)):
#   prodId[i] = []
#   cateId[i] = []
# prodId = userId
# cateId = userId

#print(prodId )

for i in range(0,len(userId)):
  prod  = list(prodOfUser[i].reshape(len(prodOfUser[i])))
  #print(prod)
  prodId[i]= prod
  cate = list(cateOfUser[i].reshape(len(cateOfUser[i])))
  cateId[i] = cate

#print(prodId)
#print(cateId)
candProd = np.array([]).astype(int)
candCate = np.array([]).astype(int)

for i in range(0,len(userId)):
  candProd = prodId[i-1]

#print(userId)

In [0]:
LastItemCatList =[]
LastItemProdList =[]

print(prodId[0])

for sList in cateId:
  LastItemCatList.append(sList[-1])
  sList.pop()

for sList in prodId:
  LastItemProdList.append(sList[-1])
  sList.pop()
  # print(LastItemCatList)

candProd=(np.array(LastItemProdList))
candCate=(np.array(LastItemCatList))

[0, 7, 29, 29, 63]


In [0]:
prodId = np.array(prodId)

for i in range(len(prodId)):
  prodId[i] = np.array(prodId[i])

print(prodId.shape)

cateId = np.array(cateId)

for i in range(len(cateId)):
  cateId[i] = np.array(cateId[i])

print(cateId.shape)

print(prodId[0].shape)
print(cateId[0].shape)

prodId = prodId.reshape(1,prodId.shape[0])
cateId = cateId.reshape(1,cateId.shape[0])
print(prodId.shape)
print(cateId.shape)

(991,)
(991,)
(4,)
(4,)
(1, 991)
(1, 991)


In [0]:
candProd = candProd.reshape(1,candProd.shape[0])
candCate = candCate.reshape(1,candCate.shape[0])

In [0]:
from keras.models import Model
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import  AveragePooling1D, Flatten, Dense, Dropout, Embedding, Lambda, Input, Concatenate, PReLU
from keras.callbacks import ModelCheckpoint
from keras.metrics import binary_accuracy 
from keras import regularizers
import tensorflow as tf

embedOutDim = 200
poolSize = 2

##################################################################
prodInput = Input(shape=(None,), name= 'products') #batch_shape(None, None)
prodEmbed =  Embedding(input_dim =numberOfProducts ,output_dim= embedOutDim, embeddings_regularizer= regularizers.l2(0.01)) (prodInput)

##################################################################
cateInput = Input(shape=(None,), name= 'categories')
cateEmbed =  Embedding(input_dim = numberOfCategories , output_dim=embedOutDim, embeddings_regularizer= regularizers.l2(0.01)) (cateInput)

##################################################################
candProdInput = Input(shape=(None,),  name= 'candidateProdId')
candProdEmbed =  Embedding(input_dim = numberOfProducts , output_dim=embedOutDim, embeddings_regularizer= regularizers.l2(0.01)) (candProdInput)


##################################################################
candCateInput = Input(shape=(None,),  name= 'candidateCateId')
candCateEmbed =  Embedding(input_dim = numberOfCategories , output_dim=embedOutDim, embeddings_regularizer= regularizers.l2(0.01)) (candCateInput)

Goods       = Concatenate()([prodEmbed,cateEmbed])
pooledGoods = AveragePooling1D(pool_size=poolSize, padding='same')(Goods)

Candidate = Concatenate()([candProdEmbed,candCateEmbed])
#print(tf.shape(Candidate))

Concatendated = Concatenate(axis=-2)([pooledGoods,Candidate])
#tf.reshape(Concatendated, (Concatendated.shape[0]*Concatendated.shape[1]*Concatendated.shape[2]))

Flattened     = Lambda(lambda x: tf.keras.backend.batch_flatten(x))(Concatendated)

Dense1  = Dense(200)(Flattened)        #(Concatendated)
Dense1  = PReLU()(Dense1)

Dense2  = Dense(80)(Dense1)
Dense2  = PReLU()(Dense2)

CTR     = Dense(1, activation='softmax',use_bias = False, name= 'ctr')(Dense2)

model = Model(inputs=[userInput, prodInput, cateInput,candProdInput, candCateInput ], outputs= [CTR])
model.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit([userId, prodId, cateId, candProd, candCate], 1, epochs = 5)
#print(userEmbed)

Using TensorFlow backend.


TypeError: ignored

#New Section

In [0]:
import pandas as pd
import gzip
import json
import numpy as np

# convert from json to pandas
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

# make dict and fit it by ids
def dict_ids(col):
  id = 0
  dic = dict() 
  for i in col:
    if i in dic:
      dic[i] = dic.get(i)
    else:
      dic[i] = id
      id += 1
  return dic

# make new col by mapping last ids to new ids
def new_col(col,dic):
  lis = list()
  for i in col:
    #if i in dic:
    lis.append(dic.get(i))
  return lis

df_all_beauty = getDF('/content/drive/My Drive/All Beauty/All_Beauty_5.json.gz')
#df_amazon_fashion = getDF('/content/drive/My Drive/Amazon Fashion/AMAZON_FASHION_5.json.gz')

"""#df_appliances = getDF('/content/drive/My Drive/Appliances/Appliances_5.json.gz')
df_books = getDF('/content/drive/My Drive/Books/Books_5.json.gz')
df_cell_phones = getDF('/content/drive/My Drive/Cell Phones and Accessories/Cell_Phones_and_Accessories_5.json.gz')
df_digital_music = getDF('/content/drive/My Drive/Digital Music/Copy of Digital_Music_5.json.gz') 
df_electronics = getDF('/content/drive/My Drive/Electronics/Copy of Electronics_5.json.gz')
df_musical_instruments = getDF('/content/drive/My Drive/Musical Instruments/Copy of Musical_Instruments_5.json.gz')
df_software = getDF('/content/drive/My Drive/Software/Copy of Software_5.json.gz')
df_video_games = getDF('/content/drive/My Drive/Video Games/Video_Games_5.json.gz')
"""
#userId = np.array(df_all_beauty['reviewerID'])

dict_all_beauty = dict_ids(userId)
# dict_amazon_fashion = dict_ids(df_amazon_fashion['reviewerID'])
"""#dict_appliances = dict_ids(df_appliances['reviewerID'])
dict_books = dict_ids(df_books['reviewerID'])
dict_cell_phones = dict_ids(cell_phones['reviewerID'])
dict_digital_music = dict_ids(df_digital_music['reviewerID'])
dict_electronics = dict_ids(df_electronics['reviewerID'])
dict_musical_instruments = dict_ids(df_musical_instruments['reviewerID'])
dict_software = dict_ids(df_software['reviewerID'])
dict_video_games = dict_ids(df_video_games['reviewerID'])
"""
#productId = np.array(df_all_beauty['asin'])

dict_user_all_beauty = dict_ids(df_all_beauty['asin'])
dict_user_amazon_fashion = dict_ids(df_amazon_fashion['asin'])
"""#dict_user_appliances = dict_ids(df_appliances['asin'])
dict_user_books = dict_ids(df_books['asin'])
dict_user_cell_phones = dict_ids(cell_phones['asin'])
dict_user_digital_music = dict_ids(df_digital_music['asin'])
dict_user_electronics = dict_ids(df_electronics['asin'])
dict_user_musical_instruments = dict_ids(df_musical_instruments['asin'])
dict_user_software = dict_ids(df_software['asin'])
dict_user_video_games = dict_ids(df_video_games['asin'])
"""
df_all_beauty['userID']=new_col(df_all_beauty['reviewerID'],dict_all_beauty)
#df_amazon_fashion['userID']=new_col(df_amazon_fashion['reviewerID'],dict_amazon_fashion)
"""#df_appliances['userID']=new_col(df_appliances['reviewerID'],dict_appliances)
df_books['userID']=new_col(df_books['reviewerID'],dict_books)
df_cell_phones['userID']=new_col(df_cell_phones['reviewerID'],dict_cell_phones)
df_digital_music['userID']=new_col(df_digital_music['reviewerID'],dict_digital_music)
df_electronics['userID']=new_col(df_electronics['reviewerID'],dict_electronics)
df_musical_instruments['userID']=new_col(df_musical_instruments['reviewerID'],dict_musical_instruments)
df_software['userID']=new_col(df_software['reviewerID'],dict_software)
df_video_games['userID']=new_col(df_video_games['reviewerID'],dict_video_games)
"""
df_all_beauty['productID']=new_col(df_all_beauty['asin'],dict_user_all_beauty)
#df_amazon_fashion['productID']=new_col(df_amazon_fashion['asin'],dict_user_amazon_fashion)
"""#df_appliances['productID']=new_col(df_appliances['asin'],dict_user_appliances)
df_books['productID']=new_col(df_books['asin'],dict_user_books)
df_cell_phones['productID']=new_col(df_cell_phones['asin'],dict_user_cell_phones)
df_digital_music['productID']=new_col(df_digital_music['asin'],dict_user_digital_music)
df_electronics['productID']=new_col(df_electronics['asin'],dict_user_electronics)
df_musical_instruments['productID']=new_col(df_musical_instruments['asin'],dict_user_musical_instruments)
df_software['productID']=new_col(df_software['asin'],dict_user_software)
df_video_games['productID']=new_col(df_video_games['asin'],dict_user_video_games)
"""
# this function  not finished 
def listofproduct(productID,userID):
  dic = dict()
  for i in range(0,len(userID)):
    dic[userID[i]]
  return dic  


print(df_all_beauty['reviewerID'])
print(df_all_beauty['userID'])
#print(df_amazon_fashion)
# print(listofproduct(df_all_beauty['productID'],df_all_beauty['userID']))

#Implementations

Implementation 1

In [0]:
# -*- coding:utf-8 -*-
"""
Author:
    Weichen Shen,wcshen1994@163.com

Reference:
    [1] Zhou G, Zhu X, Song C, et al. Deep interest network for click-through rate prediction[C]//Proceedings of the 24th ACM SIGKDD International Conference on Knowledge Discovery & Data Mining. ACM, 2018: 1059-1068. (https://arxiv.org/pdf/1706.06978.pdf)
"""

from tensorflow.python.keras.layers import Dense,Concatenate, Flatten
from tensorflow.python.keras.models import Model

from ..inputs import  build_input_features,create_embedding_matrix,SparseFeat,VarLenSparseFeat,DenseFeat,embedding_lookup,get_dense_input,varlen_embedding_lookup,get_varlen_pooling_list,combined_dnn_input
from ..layers.core import DNN, PredictionLayer
from ..layers.sequence import AttentionSequencePoolingLayer
from ..layers.utils import concat_func, NoMask


def DIN(dnn_feature_columns, history_feature_list, dnn_use_bn=False,
        dnn_hidden_units=(200, 80), dnn_activation='relu', att_hidden_size=(80, 40), att_activation="dice",
        att_weight_normalization=False, l2_reg_dnn=0, l2_reg_embedding=1e-6, dnn_dropout=0, init_std=0.0001, seed=1024,
        task='binary'):
    """Instantiates the Deep Interest Network architecture.

    :param dnn_feature_columns: An iterable containing all the features used by deep part of the model.
    :param history_feature_list: list,to indicate  sequence sparse field
    :param dnn_use_bn: bool. Whether use BatchNormalization before activation or not in deep net
    :param dnn_hidden_units: list,list of positive integer or empty list, the layer number and units in each layer of deep net
    :param dnn_activation: Activation function to use in deep net
    :param att_hidden_size: list,list of positive integer , the layer number and units in each layer of attention net
    :param att_activation: Activation function to use in attention net
    :param att_weight_normalization: bool.Whether normalize the attention score of local activation unit.
    :param l2_reg_dnn: float. L2 regularizer strength applied to DNN
    :param l2_reg_embedding: float. L2 regularizer strength applied to embedding vector
    :param dnn_dropout: float in [0,1), the probability we will drop out a given DNN coordinate.
    :param init_std: float,to use as the initialize std of embedding vector
    :param seed: integer ,to use as random seed.
    :param task: str, ``"binary"`` for  binary logloss or  ``"regression"`` for regression loss
    :return: A Keras model instance.

    """


    features = build_input_features(dnn_feature_columns)

    sparse_feature_columns = list(filter(lambda x:isinstance(x,SparseFeat),dnn_feature_columns)) if dnn_feature_columns else []
    dense_feature_columns = list(
        filter(lambda x: isinstance(x, DenseFeat), dnn_feature_columns)) if dnn_feature_columns else []
    varlen_sparse_feature_columns = list(filter(lambda x: isinstance(x, VarLenSparseFeat), dnn_feature_columns)) if dnn_feature_columns else []


    history_feature_columns = []
    sparse_varlen_feature_columns = []
    history_fc_names = list(map(lambda x: "hist_" + x, history_feature_list))
    for fc in varlen_sparse_feature_columns:
        feature_name = fc.name
        if feature_name in history_fc_names:
            history_feature_columns.append(fc)
        else:
            sparse_varlen_feature_columns.append(fc)


    inputs_list = list(features.values())


    embedding_dict = create_embedding_matrix(dnn_feature_columns, l2_reg_embedding, init_std, seed, prefix="")


    query_emb_list = embedding_lookup(embedding_dict, features, sparse_feature_columns, history_feature_list,
                                      history_feature_list,to_list=True)
    keys_emb_list = embedding_lookup(embedding_dict, features, history_feature_columns, history_fc_names,
                                     history_fc_names,to_list=True)
    dnn_input_emb_list = embedding_lookup(embedding_dict, features, sparse_feature_columns,
                                          mask_feat_list=history_feature_list,to_list=True)
    dense_value_list = get_dense_input(features, dense_feature_columns)

    sequence_embed_dict = varlen_embedding_lookup(embedding_dict,features,sparse_varlen_feature_columns)
    sequence_embed_list = get_varlen_pooling_list(sequence_embed_dict, features, sparse_varlen_feature_columns,to_list=True)

    dnn_input_emb_list += sequence_embed_list


    keys_emb = concat_func(keys_emb_list, mask=True)
    deep_input_emb = concat_func(dnn_input_emb_list)
    query_emb = concat_func(query_emb_list, mask=True)
    hist = AttentionSequencePoolingLayer(att_hidden_size, att_activation,
                                         weight_normalization=att_weight_normalization, supports_masking=True)([
        query_emb, keys_emb])

    deep_input_emb = Concatenate()([NoMask()(deep_input_emb), hist])
    deep_input_emb = Flatten()(deep_input_emb)
    dnn_input = combined_dnn_input([deep_input_emb],dense_value_list)
    output = DNN(dnn_hidden_units, dnn_activation, l2_reg_dnn,
                 dnn_dropout, dnn_use_bn, seed)(dnn_input)
    final_logit = Dense(1, use_bias=False)(output)

    output = PredictionLayer(task)(final_logit)

    model = Model(inputs=inputs_list, outputs=output)
    return model


Implementation 2



In [0]:
import tensorflow as tf

from Dice import dice

class Model(object):

  def __init__(self, user_count, item_count, cate_count, cate_list, predict_batch_size, predict_ads_num):

    self.u = tf.placeholder(tf.int32, [None,]) # [B]
    self.i = tf.placeholder(tf.int32, [None,]) # [B]
    self.j = tf.placeholder(tf.int32, [None,]) # [B]
    self.y = tf.placeholder(tf.float32, [None,]) # [B]
    self.hist_i = tf.placeholder(tf.int32, [None, None]) # [B, T]
    self.sl = tf.placeholder(tf.int32, [None,]) # [B]
    self.lr = tf.placeholder(tf.float64, [])

    hidden_units = 128

    user_emb_w = tf.get_variable("user_emb_w", [user_count, hidden_units])
    item_emb_w = tf.get_variable("item_emb_w", [item_count, hidden_units // 2])
    item_b = tf.get_variable("item_b", [item_count],
                             initializer=tf.constant_initializer(0.0))
    cate_emb_w = tf.get_variable("cate_emb_w", [cate_count, hidden_units // 2])
    cate_list = tf.convert_to_tensor(cate_list, dtype=tf.int64)

    ic = tf.gather(cate_list, self.i)
    i_emb = tf.concat(values = [
        tf.nn.embedding_lookup(item_emb_w, self.i),
        tf.nn.embedding_lookup(cate_emb_w, ic),
        ], axis=1)
    i_b = tf.gather(item_b, self.i)

    jc = tf.gather(cate_list, self.j)
    j_emb = tf.concat([
        tf.nn.embedding_lookup(item_emb_w, self.j),
        tf.nn.embedding_lookup(cate_emb_w, jc),
        ], axis=1)
    j_b = tf.gather(item_b, self.j)

    hc = tf.gather(cate_list, self.hist_i)
    h_emb = tf.concat([
        tf.nn.embedding_lookup(item_emb_w, self.hist_i),
        tf.nn.embedding_lookup(cate_emb_w, hc),
        ], axis=2)

    hist_i =attention(i_emb, h_emb, self.sl)
    #-- attention end ---
    
    hist_i = tf.layers.batch_normalization(inputs = hist_i)
    hist_i = tf.reshape(hist_i, [-1, hidden_units], name='hist_bn')
    hist_i = tf.layers.dense(hist_i, hidden_units, name='hist_fcn')

    u_emb_i = hist_i
    
    hist_j =attention(j_emb, h_emb, self.sl)
    #-- attention end ---
    
    hist_j = tf.layers.batch_normalization(inputs = hist_j)
    hist_j = tf.reshape(hist_j, [-1, hidden_units], name='hist_bn')
    hist_j = tf.layers.dense(hist_j, hidden_units, name='hist_fcn', reuse=True)

    u_emb_j = hist_j
    print u_emb_i.get_shape().as_list()
    print u_emb_j.get_shape().as_list()
    print i_emb.get_shape().as_list()
    print j_emb.get_shape().as_list()
    #-- fcn begin -------
    din_i = tf.concat([u_emb_i, i_emb], axis=-1)
    din_i = tf.layers.batch_normalization(inputs=din_i, name='b1')
    d_layer_1_i = tf.layers.dense(din_i, 80, activation=tf.nn.sigmoid, name='f1')
    #if u want try dice change sigmoid to None and add dice layer like following two lines. You can also find model_dice.py in this folder.
    # d_layer_1_i = tf.layers.dense(din_i, 80, activation=None, name='f1')
    # d_layer_1_i = dice(d_layer_1_i, name='dice_1')
    # d_layer_2_i = tf.layers.dense(d_layer_1_i, 40, activation=None, name='f2')
    d_layer_2_i = tf.layers.dense(d_layer_1_i, 40, activation=tf.nn.sigmoid, name='f2')
    #d_layer_2_i = dice(d_layer_2_i, name='dice_2')
    d_layer_3_i = tf.layers.dense(d_layer_2_i, 1, activation=None, name='f3')
    din_j = tf.concat([u_emb_j, j_emb], axis=-1)
    din_j = tf.layers.batch_normalization(inputs=din_j, name='b1', reuse=True)
    d_layer_1_j = tf.layers.dense(din_j, 80, activation=tf.nn.sigmoid, name='f1', reuse=True)
    # d_layer_1_j = tf.layers.dense(din_j, 80, activation=None, name='f1', reuse=True)
    # d_layer_1_j = dice(d_layer_1_j, name='dice_1')
    d_layer_2_j = tf.layers.dense(d_layer_1_j, 40, activation=tf.nn.sigmoid, name='f2', reuse=True)
    # d_layer_2_j = tf.layers.dense(d_layer_1_j, 40, activation=None, name='f2', reuse=True)
    # d_layer_2_j = dice(d_layer_2_j, name='dice_2')
    d_layer_3_j = tf.layers.dense(d_layer_2_j, 1, activation=None, name='f3', reuse=True)
    d_layer_3_i = tf.reshape(d_layer_3_i, [-1])
    d_layer_3_j = tf.reshape(d_layer_3_j, [-1])
    x = i_b - j_b + d_layer_3_i - d_layer_3_j # [B]
    self.logits = i_b + d_layer_3_i
    
    # prediciton for selected items
    # logits for selected item:
    item_emb_all = tf.concat([
        item_emb_w,
        tf.nn.embedding_lookup(cate_emb_w, cate_list)
        ], axis=1)
    item_emb_sub = item_emb_all[:predict_ads_num,:]
    item_emb_sub = tf.expand_dims(item_emb_sub, 0)
    item_emb_sub = tf.tile(item_emb_sub, [predict_batch_size, 1, 1])
    hist_sub =attention_multi_items(item_emb_sub, h_emb, self.sl)
    #-- attention end ---
    
    hist_sub = tf.layers.batch_normalization(inputs = hist_sub, name='hist_bn', reuse=tf.AUTO_REUSE)
    # print hist_sub.get_shape().as_list() 
    hist_sub = tf.reshape(hist_sub, [-1, hidden_units])
    hist_sub = tf.layers.dense(hist_sub, hidden_units, name='hist_fcn', reuse=tf.AUTO_REUSE)

    u_emb_sub = hist_sub
    item_emb_sub = tf.reshape(item_emb_sub, [-1, hidden_units])
    din_sub = tf.concat([u_emb_sub, item_emb_sub], axis=-1)
    din_sub = tf.layers.batch_normalization(inputs=din_sub, name='b1', reuse=True)
    d_layer_1_sub = tf.layers.dense(din_sub, 80, activation=tf.nn.sigmoid, name='f1', reuse=True)
    #d_layer_1_sub = dice(d_layer_1_sub, name='dice_1_sub')
    d_layer_2_sub = tf.layers.dense(d_layer_1_sub, 40, activation=tf.nn.sigmoid, name='f2', reuse=True)
    #d_layer_2_sub = dice(d_layer_2_sub, name='dice_2_sub')
    d_layer_3_sub = tf.layers.dense(d_layer_2_sub, 1, activation=None, name='f3', reuse=True)
    d_layer_3_sub = tf.reshape(d_layer_3_sub, [-1, predict_ads_num])
    self.logits_sub = tf.sigmoid(item_b[:predict_ads_num] + d_layer_3_sub)
    self.logits_sub = tf.reshape(self.logits_sub, [-1, predict_ads_num, 1])
    #-- fcn end -------

    
    self.mf_auc = tf.reduce_mean(tf.to_float(x > 0))
    self.score_i = tf.sigmoid(i_b + d_layer_3_i)
    self.score_j = tf.sigmoid(j_b + d_layer_3_j)
    self.score_i = tf.reshape(self.score_i, [-1, 1])
    self.score_j = tf.reshape(self.score_j, [-1, 1])
    self.p_and_n = tf.concat([self.score_i, self.score_j], axis=-1)
    print self.p_and_n.get_shape().as_list()


    # Step variable
    self.global_step = tf.Variable(0, trainable=False, name='global_step')
    self.global_epoch_step = \
        tf.Variable(0, trainable=False, name='global_epoch_step')
    self.global_epoch_step_op = \
        tf.assign(self.global_epoch_step, self.global_epoch_step+1)

    self.loss = tf.reduce_mean(
        tf.nn.sigmoid_cross_entropy_with_logits(
            logits=self.logits,
            labels=self.y)
        )

    trainable_params = tf.trainable_variables()
    self.opt = tf.train.GradientDescentOptimizer(learning_rate=self.lr)
    gradients = tf.gradients(self.loss, trainable_params)
    clip_gradients, _ = tf.clip_by_global_norm(gradients, 5)
    self.train_op = self.opt.apply_gradients(
        zip(clip_gradients, trainable_params), global_step=self.global_step)


  def train(self, sess, uij, l):
    loss, _ = sess.run([self.loss, self.train_op], feed_dict={
        self.u: uij[0],
        self.i: uij[1],
        self.y: uij[2],
        self.hist_i: uij[3],
        self.sl: uij[4],
        self.lr: l,
        })
    return loss

  def eval(self, sess, uij):
    u_auc, socre_p_and_n = sess.run([self.mf_auc, self.p_and_n], feed_dict={
        self.u: uij[0],
        self.i: uij[1],
        self.j: uij[2],
        self.hist_i: uij[3],
        self.sl: uij[4],
        })
    return u_auc, socre_p_and_n
  
  def test(self, sess, uij):
    return sess.run(self.logits_sub, feed_dict={
        self.u: uij[0],
        self.i: uij[1],
        self.j: uij[2],
        self.hist_i: uij[3],
        self.sl: uij[4],
        })
  

  def save(self, sess, path):
    saver = tf.train.Saver()
    saver.save(sess, save_path=path)

  def restore(self, sess, path):
    saver = tf.train.Saver()
    saver.restore(sess, save_path=path)

def extract_axis_1(data, ind):
  batch_range = tf.range(tf.shape(data)[0])
  indices = tf.stack([batch_range, ind], axis=1)
  res = tf.gather_nd(data, indices)
  return res

def attention(queries, keys, keys_length):
  '''
    queries:     [B, H]
    keys:        [B, T, H]
    keys_length: [B]
  '''
  queries_hidden_units = queries.get_shape().as_list()[-1]
  queries = tf.tile(queries, [1, tf.shape(keys)[1]])
  queries = tf.reshape(queries, [-1, tf.shape(keys)[1], queries_hidden_units])
  din_all = tf.concat([queries, keys, queries-keys, queries*keys], axis=-1)
  d_layer_1_all = tf.layers.dense(din_all, 80, activation=tf.nn.sigmoid, name='f1_att', reuse=tf.AUTO_REUSE)
  d_layer_2_all = tf.layers.dense(d_layer_1_all, 40, activation=tf.nn.sigmoid, name='f2_att', reuse=tf.AUTO_REUSE)
  d_layer_3_all = tf.layers.dense(d_layer_2_all, 1, activation=None, name='f3_att', reuse=tf.AUTO_REUSE)
  d_layer_3_all = tf.reshape(d_layer_3_all, [-1, 1, tf.shape(keys)[1]])
  outputs = d_layer_3_all 
  # Mask
  key_masks = tf.sequence_mask(keys_length, tf.shape(keys)[1])   # [B, T]
  key_masks = tf.expand_dims(key_masks, 1) # [B, 1, T]
  paddings = tf.ones_like(outputs) * (-2 ** 32 + 1)
  outputs = tf.where(key_masks, outputs, paddings)  # [B, 1, T]

  # Scale
  outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)

  # Activation
  outputs = tf.nn.softmax(outputs)  # [B, 1, T]

  # Weighted sum
  outputs = tf.matmul(outputs, keys)  # [B, 1, H]

  return outputs

def attention_multi_items(queries, keys, keys_length):
  '''
    queries:     [B, N, H] N is the number of ads
    keys:        [B, T, H] 
    keys_length: [B]
  '''
  queries_hidden_units = queries.get_shape().as_list()[-1]
  queries_nums = queries.get_shape().as_list()[1]
  queries = tf.tile(queries, [1, 1, tf.shape(keys)[1]])
  queries = tf.reshape(queries, [-1, queries_nums, tf.shape(keys)[1], queries_hidden_units]) # shape : [B, N, T, H]
  max_len = tf.shape(keys)[1]
  keys = tf.tile(keys, [1, queries_nums, 1])
  keys = tf.reshape(keys, [-1, queries_nums, max_len, queries_hidden_units]) # shape : [B, N, T, H]
  din_all = tf.concat([queries, keys, queries-keys, queries*keys], axis=-1)
  d_layer_1_all = tf.layers.dense(din_all, 80, activation=tf.nn.sigmoid, name='f1_att', reuse=tf.AUTO_REUSE)
  d_layer_2_all = tf.layers.dense(d_layer_1_all, 40, activation=tf.nn.sigmoid, name='f2_att', reuse=tf.AUTO_REUSE)
  d_layer_3_all = tf.layers.dense(d_layer_2_all, 1, activation=None, name='f3_att', reuse=tf.AUTO_REUSE)
  d_layer_3_all = tf.reshape(d_layer_3_all, [-1, queries_nums, 1, max_len])
  outputs = d_layer_3_all 
  # Mask
  key_masks = tf.sequence_mask(keys_length, max_len)   # [B, T]
  key_masks = tf.tile(key_masks, [1, queries_nums])
  key_masks = tf.reshape(key_masks, [-1, queries_nums, 1, max_len]) # shape : [B, N, 1, T]
  paddings = tf.ones_like(outputs) * (-2 ** 32 + 1)
  outputs = tf.where(key_masks, outputs, paddings)  # [B, N, 1, T]

  # Scale
  outputs = outputs / (keys.get_shape().as_list()[-1] ** 0.5)

  # Activation
  outputs = tf.nn.softmax(outputs)  # [B, N, 1, T]
  outputs = tf.reshape(outputs, [-1, 1, max_len])
  keys = tf.reshape(keys, [-1, max_len, queries_hidden_units])
  #print outputs.get_shape().as_list()
  #print keys.get_sahpe().as_list()
  # Weighted sum
  outputs = tf.matmul(outputs, keys)
  outputs = tf.reshape(outputs, [-1, queries_nums, queries_hidden_units])  # [B, N, 1, H]
  print outputs.get_shape().as_list()
  return outputs
